<a href="https://colab.research.google.com/github/u-dharshini/CodeAlpha.Simple-E-commerce-Store-/blob/main/life_decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
from collections import Counter

# -------------------------------
# Dataset
# -------------------------------
data = [
    {"StudyHours": "High",   "Attendance": "High", "Result": "Pass"},
    {"StudyHours": "High",   "Attendance": "Low",  "Result": "Pass"},
    {"StudyHours": "Medium", "Attendance": "High", "Result": "Pass"},
    {"StudyHours": "Low",    "Attendance": "High", "Result": "Pass"},
    {"StudyHours": "Low",    "Attendance": "Low",  "Result": "Fail"},
    {"StudyHours": "Medium", "Attendance": "Low",  "Result": "Fail"},
    {"StudyHours": "Low",    "Attendance": "High", "Result": "Fail"},
]

# -------------------------------
# Entropy
# -------------------------------
def entropy(dataset):
    counts = Counter(row["Result"] for row in dataset)
    total = len(dataset)

    ent = 0
    for c in counts.values():
        p = c / total
        ent -= p * math.log2(p)

    return ent

# -------------------------------
# Information Gain
# -------------------------------
def information_gain(dataset, attribute):
    total_entropy = entropy(dataset)
    total = len(dataset)

    weighted_entropy = 0
    for value in set(row[attribute] for row in dataset):
        subset = [row for row in dataset if row[attribute] == value]
        weighted_entropy += (len(subset) / total) * entropy(subset)

    return total_entropy - weighted_entropy

# -------------------------------
# Majority class (with tie handling)
# -------------------------------
def majority_class(dataset):
    counts = Counter(row["Result"] for row in dataset)
    return counts.most_common(1)[0][0]

# -------------------------------
# Build ID3 Decision Tree
# -------------------------------
def build_tree(dataset, attributes, parent_majority=None):
    results = [row["Result"] for row in dataset]
    counts = Counter(results)

    # Pure node
    if len(counts) == 1:
        return results[0]

    # No attributes left → use parent majority
    if not attributes:
        return parent_majority

    # Parent majority for children
    current_majority = counts.most_common(1)[0][0]

    # Choose best attribute
    best_attr = max(attributes, key=lambda a: information_gain(dataset, a))
    tree = {best_attr: {}}

    remaining_attrs = [a for a in attributes if a != best_attr]

    for value in set(row[best_attr] for row in dataset):
        subset = [row for row in dataset if row[best_attr] == value]

        if not subset:
            tree[best_attr][value] = current_majority
        else:
            tree[best_attr][value] = build_tree(
                subset,
                remaining_attrs,
                current_majority
            )

    return tree

# -------------------------------
# Prediction
# -------------------------------
def predict(tree, sample):
    if isinstance(tree, str):
        return tree

    attribute = next(iter(tree))
    value = sample.get(attribute)

    return predict(tree[attribute][value], sample)

# -------------------------------
# Execution
# -------------------------------
attributes = ["StudyHours", "Attendance"]

print("Entropy of dataset:", entropy(data))
print("IG(StudyHours):", information_gain(data, "StudyHours"))
print("IG(Attendance):", information_gain(data, "Attendance"))

tree = build_tree(data, attributes)
print("\nDecision Tree:")
print(tree)

new_student = {"StudyHours": "Low", "Attendance": "High"}
print("\nNew Student:", new_student)
print("Predicted Result:", predict(tree, new_student))

Entropy of dataset: 0.9852281360342515
IG(StudyHours): 0.3059584928680418
IG(Attendance): 0.12808527889139443

Decision Tree:
{'StudyHours': {'Medium': {'Attendance': {'Low': 'Fail', 'High': 'Pass'}}, 'Low': {'Attendance': {'Low': 'Fail', 'High': 'Fail'}}, 'High': 'Pass'}}

New Student: {'StudyHours': 'Low', 'Attendance': 'High'}
Predicted Result: Fail
